In [4]:
# https://dataverse.harvard.edu/api/access/datafile/:persistentId?persistentId=doi:10.7910/DVN/HG7NV7/XTPZZY
# https://dataverse.harvard.edu/api/access/datafile/:persistentId?persistentId=doi:10.7910/DVN/HG7NV7/IP6BL3
# https://dataverse.harvard.edu/api/access/datafile/:persistentId?persistentId=doi:10.7910/DVN/HG7NV7/3NOQ6Q

In [5]:
# pip install ipython-sql

In [ ]:
%load_ext sql

In [7]:
import csv, sqlite3
import pandas as pd

In [8]:
con = sqlite3.connect("yash_1999.db")
cur = con.cursor()

In [ ]:
df_all = pd.read_csv('1999 (1).csv.bz2')
df_all.shape # (5527884, 29)

In [73]:
%sql sqlite:///airline.db

In [74]:
# reading 10000 records from 1999_year file
df = df_all.sample(10000)
df.to_sql('airline_1999', con, index=False, if_exists='replace')

10000

In [75]:
# add airport table
df1 = pd.read_csv('airports.csv')
df1.to_sql('airport', con, index=False, if_exists='replace')

3376

In [76]:
# add carrier table
df2 = pd.read_csv('carriers.csv')
df2.to_sql('carrier', con, index=False, if_exists='replace')

1491

In [77]:
%sql SELECT * FROM airline_1999 LIMIT 5;

 * sqlite:///airline.db
Done.


Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
1999,7,23,5,1404.0,1400,1518.0,1517,US,1278,N433US,74.0,77.0,60.0,1.0,4.0,PIT,BDL,406,4,10,0,None,0,None,None,None,None,None
1999,2,11,4,841.0,845,1027.0,1036,TW,485,N24343,166.0,171.0,134.0,-9.0,-4.0,MIA,STL,1068,22,10,0,None,0,None,None,None,None,None
1999,3,10,3,1441.0,1425,1702.0,1649,AA,2020,N539AA,141.0,144.0,125.0,13.0,16.0,AUS,ORD,978,6,10,0,None,0,None,None,None,None,None
1999,2,9,2,1156.0,1200,1244.0,1259,UA,617,N815UA,108.0,119.0,96.0,-15.0,-4.0,DCA,ORD,612,4,8,0,None,0,None,None,None,None,None
1999,12,28,2,1534.0,1412,1640.0,1536,AA,1227,N454AA,186.0,204.0,167.0,64.0,82.0,DFW,LAX,1235,5,14,0,None,0,None,None,None,None,None


In [175]:
%sql SELECT * FROM airport LIMIT 5

 * sqlite:///airline.db
Done.


iata,airport,city,state,country,lat,long
00M,Thigpen,Bay Springs,MS,USA,31.95376472,-89.23450472
00R,Livingston Municipal,Livingston,TX,USA,30.68586111,-95.01792778
00V,Meadow Lake,Colorado Springs,CO,USA,38.94574889,-104.5698933
01G,Perry-Warsaw,Perry,NY,USA,42.74134667,-78.05208056
01J,Hilliard Airpark,Hilliard,FL,USA,30.6880125,-81.90594389


In [182]:
%sql SELECT * FROM carrier WHERE Code = 'WL';


 * sqlite:///airline.db
Done.


Code,Description
WL,World Air Network


airplane fly from Origin which has DepDelay to Dest which has ArrDelay

delay time for airports is calculated as total delay due to ArrDelay & DepDelay

the three airports with the highest delay time (in hours).

In [179]:
%%sql
SELECT left.Airport_Code,
    right.airport AS Airport_Name,
    left.Arrival_Delay,
    left.Departure_Delay,
    left.Total_Delay    
FROM
    (SELECT
        DESTINATION.Arrival_Airport AS Airport_Code,
        DESTINATION.Arrival_Delay,
        DEPARTURE.Departure_Delay, 
        SUM(DESTINATION.Arrival_Delay + DEPARTURE.Departure_Delay) AS Total_Delay
    FROM 
        (SELECT
            a.Dest AS Arrival_Airport,
            SUM(
                IIF
                (a.ArrDelay < 0,
                    0,
                    a.ArrDelay)) AS Arrival_Delay
        FROM airline_1999 AS a
        GROUP BY a.Dest) AS DESTINATION
    FULL OUTER JOIN 
        (SELECT
            a.Origin AS Departure_Airport,
            SUM(
                IIF
                (a.DepDelay < 0,
                    0,
                    a.DepDelay)) AS Departure_Delay
        FROM airline_1999 AS a
        GROUP BY a.Origin) AS DEPARTURE
    ON DESTINATION.Arrival_Airport = DEPARTURE.Departure_Airport
    GROUP BY DESTINATION.Arrival_Airport
    ORDER BY Total_Delay desc
    LIMIT 3) AS left
LEFT JOIN airport AS right
ON left.Airport_Code = right.iata;

 * sqlite:///airline.db
Done.


Airport_Code,Airport_Name,Arrival_Delay,Departure_Delay,Total_Delay
ORD,Chicago O'Hare International,6729.0,7217.0,13946.0
ATL,William B Hartsfield-Atlanta Intl,5572.0,5262.0,10834.0
DFW,Dallas-Fort Worth International,5277.0,3670.0,8947.0


In [172]:
%%sql
SELECT
    left.Carrier_Code,
    right.Description,
    left.Arrival_Delay,
    left.Departure_Delay,
    (left.Arrival_Delay+left.Departure_Delay) AS Total_Delay,
    MAX(left.Arrival_Delay,left.Departure_Delay) AS Largest_Delay,
    IIF(
        MAX(left.Arrival_Delay,left.Departure_Delay) = left.Arrival_Delay,
            'Arrival_Delay',
            'Departure_Delay') AS Delay_Type 
FROM
    (SELECT
        a.UniqueCarrier AS Carrier_Code, 
        SUM(
            IIF
            (a.ArrDelay < 0,
                0,
                a.ArrDelay)) AS Arrival_Delay,
        SUM(
            IIF
            (a.DepDelay < 0,
                0,
                a.DepDelay)) AS Departure_Delay
    FROM
        airline_1999 AS a      
    GROUP BY a.UniqueCarrier) AS left
INNER JOIN carrier AS right
ON left.Carrier_Code = right.Code
ORDER BY Total_Delay desc 
LIMIT 3

 * sqlite:///airline.db
Done.


Carrier_Code,Description,Arrival_Delay,Departure_Delay,Total_Delay,Largest_Delay,Delay_Type
UA,United Air Lines Inc.,18896.0,16052.0,34948.0,18896.0,Arrival_Delay
US,US Airways Inc. (Merged with America West 9/05. Reporting for both starting 10/07.),17111.0,14327.0,31438.0,17111.0,Arrival_Delay
WN,Southwest Airlines Co.,15523.0,15454.0,30977.0,15523.0,Arrival_Delay


In [180]:
%%sql
SELECT
    a.Dest AS Arrival_Airport,
    SUM(
        IF
        (a.ArrDelay < 0,
            0,
            a.ArrDelay)) AS Arrival_Delay
FROM airline_1999 AS a
GROUP BY a.Dest;

 * sqlite:///airline.db
(sqlite3.OperationalError) no such function: IF
[SQL: SELECT
    a.Dest AS Arrival_Airport,
    SUM(
        IF
        (a.ArrDelay < 0,
            0,
            a.ArrDelay)) AS Arrival_Delay
FROM airline_1999 AS a
GROUP BY a.Dest;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
